In [1]:
%load_ext dotenv
%dotenv

In [4]:
from langchain_together import ChatTogether
import os

llm = ChatTogether(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key= os.getenv("TOGETHER_API_KEY")
)

In [5]:
ai_msg = llm.invoke('''I love programming. suggest me some good programming languages to learn.''')
print(ai_msg.content)

With so many programming languages out there, it can be overwhelming to decide which ones to learn. Here are some popular and in-demand programming languages that are worth considering:

**For Beginners:**

1. **Python**: Easy to learn, versatile, and widely used in data science, machine learning, web development, and more.
2. **JavaScript**: Essential for web development, and also used in mobile and desktop app development, game development, and server-side programming.
3. **HTML/CSS**: Not a programming language, but essential for web development and a great starting point for beginners.

**For Web Development:**

1. **Java**: Popular for Android app development, web development, and enterprise software development.
2. **Ruby**: Known for its simplicity and ease of use, popular for web development with the Ruby on Rails framework.
3. **PHP**: Widely used for web development, especially for building dynamic websites and web applications.

**For Mobile App Development:**

1. **Swift**:

In [4]:
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser

In [5]:
str_output_parser = StrOutputParser()

response_parsed = str_output_parser.invoke(ai_msg)
response_parsed

"With so many programming languages out there, it can be overwhelming to decide which ones to learn. Here are some popular and in-demand programming languages that are worth considering:\n\n**For Beginners:**\n\n1. **Python**: Easy to learn, versatile, and widely used in data science, machine learning, web development, and automation.\n2. **JavaScript**: Essential for web development, JavaScript is also used in mobile and desktop app development, game development, and server-side programming.\n3. **HTML/CSS**: Not a programming language, but essential for web development, HTML (Hypertext Markup Language) and CSS (Cascading Style Sheets) are used to create web pages and applications.\n\n**For Web Development:**\n\n1. **Java**: A popular language for Android app development, web development, and enterprise software development.\n2. **Ruby**: Known for its simplicity and ease of use, Ruby is a great language for web development, especially with the Ruby on Rails framework.\n3. **PHP**: A 

In [6]:
list_output_parser = CommaSeparatedListOutputParser()

response_parsed = list_output_parser.invoke(ai_msg)

print(response_parsed)

print(list_output_parser.get_format_instructions())

['With so many programming languages out there', 'it can be overwhelming to decide which ones to learn. Here are some popular and in-demand programming languages that are worth considering:', '**For Beginners:**', '1. **Python**: Easy to learn', 'versatile', 'and widely used in data science', 'machine learning', 'web development', 'and automation.', '2. **JavaScript**: Essential for web development', 'JavaScript is also used in mobile and desktop app development', 'game development', 'and server-side programming.', '3. **HTML/CSS**: Not a programming language', 'but essential for web development', 'HTML (Hypertext Markup Language) and CSS (Cascading Style Sheets) are used to create web pages and applications.', '**For Web Development:**', '1. **Java**: A popular language for Android app development', 'web development', 'and enterprise software development.', '2. **Ruby**: Known for its simplicity and ease of use', 'Ruby is a great language for web development', 'especially with the Rub

In [4]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke about football."

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"query": joke_query})

{'setup': 'Why did the football go to the doctor?',
 'punchline': 'Because it was feeling a little deflated!'}

In [5]:
for s in chain.stream({"query": joke_query}):
    print(s)

{}
{'setup': ''}
{'setup': 'Why'}
{'setup': 'Why did'}
{'setup': 'Why did the'}
{'setup': 'Why did the football'}
{'setup': 'Why did the football go'}
{'setup': 'Why did the football go to'}
{'setup': 'Why did the football go to the'}
{'setup': 'Why did the football go to the doctor'}
{'setup': 'Why did the football go to the doctor?'}
{'setup': 'Why did the football go to the doctor?', 'punchline': ''}
{'setup': 'Why did the football go to the doctor?', 'punchline': 'Because'}
{'setup': 'Why did the football go to the doctor?', 'punchline': 'Because it'}
{'setup': 'Why did the football go to the doctor?', 'punchline': 'Because it was'}
{'setup': 'Why did the football go to the doctor?', 'punchline': 'Because it was feeling'}
{'setup': 'Why did the football go to the doctor?', 'punchline': 'Because it was feeling a'}
{'setup': 'Why did the football go to the doctor?', 'punchline': 'Because it was feeling a little'}
{'setup': 'Why did the football go to the doctor?', 'punchline': 'Becau

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate

In [14]:
chat_template = ChatPromptTemplate.from_messages([
    ('human', 
     "I've recently adopted a {pet} which is a {breed}. Could you suggest several training tips?")])

In [21]:
chain = chat_template | llm 

In [22]:
chain.invoke({"pet": "dog", "breed": "shepherd"})

AIMessage(content='Congratulations on adopting a new furry family member. German Shepherds are highly intelligent and responsive to training, making them a popular breed for many dog owners. Here are several training tips to help you get started:\n\n1. **Establish a routine**: Shepherds thrive on routine and structure. Create a schedule for feeding, exercise, and playtime to help your dog feel secure and develop good habits.\n2. **Positive reinforcement**: Reward your dog with treats, praise, and affection when they exhibit good behavior. This encourages positive reinforcement and helps build trust between you and your dog.\n3. **Start with basic commands**: Teach basic commands like "sit," "stay," "come," and "leave it." Use positive reinforcement and be consistent in your commands.\n4. **Socialization**: Socialization is crucial for German Shepherds, as they can be wary of strangers and new environments. Introduce your dog to new people, places, and experiences to help them become co

In [23]:
%%time
chain.batch([{'pet':'dog', 'breed':'shepherd'}, 
             {'pet':'dragon', 'breed':'night fury'}])

CPU times: total: 62.5 ms
Wall time: 6.45 s


[AIMessage(content='Congratulations on adopting a new furry family member. German Shepherds are highly intelligent and responsive to training, making them a popular breed for many dog owners. Here are several training tips to help you get started:\n\n1. **Establish a routine**: Shepherds thrive on routine and structure. Create a schedule for feeding, exercise, and playtime to help your dog feel secure and develop good habits.\n2. **Positive reinforcement**: Reward your dog with treats, praise, and affection when they exhibit good behavior. This encourages positive reinforcement and helps build trust between you and your dog.\n3. **Start with basic commands**: Teach basic commands like "sit," "stay," "come," and "leave it." Use positive reinforcement and be consistent in your commands.\n4. **Socialization**: Socialization is crucial for German Shepherds, as they can be wary of strangers and new environments. Introduce your dog to new people, places, and experiences to help them become c

In [24]:
%%time
chain.invoke({'pet':'dog', 'breed':'shepherd'})

CPU times: total: 31.2 ms
Wall time: 6.13 s


AIMessage(content='Congratulations on adopting a new furry family member. German Shepherds are highly intelligent and responsive to training, making them a popular breed for many dog owners. Here are several training tips to help you get started:\n\n1. **Establish a routine**: Shepherds thrive on routine and structure. Create a schedule for feeding, exercise, and playtime to help your dog feel secure and develop good habits.\n2. **Positive reinforcement**: Reward your dog with treats, praise, and affection when they exhibit good behavior. This encourages positive reinforcement and helps build trust between you and your dog.\n3. **Socialization**: Socialization is crucial for German Shepherds, as they can be naturally wary of strangers. Introduce your dog to new people, places, and experiences to help them become confident and calm in new situations.\n4. **Basic obedience commands**: Teach basic commands like "sit," "stay," "come," and "leave it." Use positive reinforcement and be consi

In [25]:
%%time
chain.invoke({'pet':'dragon', 'breed':'night fury'})

CPU times: total: 15.6 ms
Wall time: 7.56 s


AIMessage(content='Congratulations on adopting a Night Fury dragon! Training a dragon, especially a Night Fury, requires patience, consistency, and a deep understanding of their behavior and needs. Here are some tips to help you train your new companion:\n\n1. **Establish trust**: Night Furies are known for their intelligence and independence. Building trust is crucial, so start by spending quiet time with your dragon, letting it become familiar with your presence, voice, and scent.\n2. **Use positive reinforcement**: Reward your dragon with treats, praise, and affection when it responds to commands or exhibits desired behavior. Avoid punishment or negative reinforcement, as this can lead to fear and aggression.\n3. **Start with basic commands**: Teach your dragon basic commands like "sit," "stay," "come," and "leave it." Use a calm, clear tone and consistent hand signals to help your dragon understand what you want.\n4. **Respect personal space**: Night Furies value their independence